## PARITE DATA 
SONIA SAI 
ETIA-ANAELLE SAKOA

## PARTIE A : ANALYSE ET PRÉTRAITEMENT


Cette partie vise à faire une analyse exploratoire des données récoltées et une segmentation du marché boursier

# 1. Collecte des données 

In [1]:
!pip install yfinance


     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB 217.9 kB/s eta 0:00:14
     ---------------------------------------- 0.0/3.0 MB 259.2 kB/s eta 0:00:12
      --------------------------------------- 0.1/3.0 MB 290.5 kB/s eta 0:00:11
     -- ------------------------------------- 0.2/3.0 MB 1.0 MB/s eta 0:00:03
     ---- ----------------------------------- 0.4/3.0 MB 1.6 MB/s eta 0:00:02
     ------- -------------------------------- 0.6/3.0 MB 2.2 MB/s eta 0:00:02
     ---------- ----------------------------- 0.8/3.0 MB 2.3 MB/s eta 0:00:01
     ------------- -------------------------- 1.1/3.0 MB 2.9 MB/s eta 0:00:01
     ----------------- ---------------------- 1.3/3.0 MB 3.1 MB/s eta 0:00:01
     ------------------- -------------------- 1.5/3.0 MB 3.1 MB/s eta 0:00:01
     --------------------- ------------------ 1.6/3.0 MB 3.2 MB/s eta 0

In [2]:
import yfinance as yf
import pandas as pd

In [3]:
start_date = "2018-01-01"
end_date = "2024-12-31"

In [4]:
#téléchargement des données
tickers = ['TSLA', 'AAPL', 'GOOG', 'AMZN', 'META']  
data = yf.download(tickers, start=start_date, end=end_date)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  5 of 5 completed


In [5]:
data.to_csv("donnees_boursieres.csv")

# 2. Analyse et prétraitement des données 

In [6]:
# 1. Analyse exploratrice :
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

In [8]:
fichier="donnees_boursieres.csv"
df_bours=pd.read_csv(fichier, sep=";")
df_bours.head()

,"Price,Close,Close,Close,Close,Close,High,High,High,High,High,Low,Low,Low,Low,Low,Open,Open,Open,Open,Open,Volume,Volume,Volume,Volume,Volume"
0,"Ticker,AAPL,AMZN,GOOG,META,TSLA,AAPL,AMZN,GOOG..."
1,"Date,,,,,,,,,,,,,,,,,,,,,,,,,"
2,"2018-01-02,40.47983932495117,59.45050048828125..."
3,"2018-01-03,40.4727897644043,60.209999084472656..."
4,"2018-01-04,40.66078186035156,60.47949981689453..."


In [ ]:
df_raw = pd.read_csv("/mnt/data/donnees_boursieres.csv", header=[0, 1])